In [ ]:
class OptionPricing:
    def __init__(self, stockname, strike_price, ttm, option_type = 'Call', visualizaition = True):
        self.stockname = stockname
        self.option_type = option_type
        self.visual = visualization
        self.strike_price = strike_price
        self.ttm = ttm
        
    
    def get_stock_info(self, stock_name, start = 0, end = 0):

        return self.price,self.std # stock price and std
    
    
    
    def get_rf(self, date = 0, tenor = '10 yr'):

        return self.r_f #risk free rate
    
    
    def BS_model(self):

        stock_price = self.price # from get_stock_info
        var = self.std ** 2
        strike_price = self.strike_price
        r_f = self.r_f
        ttm =self.ttm
        
        return self.BS_df 

        
    
    def BS_sensitivity_plot(self):
        stock_price = self.price
        var = self.std ** 2
        strike_price = self.strike_price
        r_f = self.r_f
        ttm =self.ttm
        option_type = self.option_type
        
        
        
        
    def mc_euro_options(self, option_type,S,K,T,r,v,simulations):
        

    
    def simulations_converge(self, option_type,S,K,T,r,v,simulations):
        pass
    
    def stock_time(self, S,K,r,v,simulations,steps,time_delta):
        pass
    
    def BinomialTreeCRR(self, T, S, K, r, sigma, n, option_type='C'):
        pass